In [4]:
from tarfile import version
!pip install -U pip setuptools wheel
!pip install -U torch torchvision torchaudio

In [2]:
!git clone https://github.com/myshell-ai/MeloTTS.git

Cloning into 'MeloTTS'...


In [2]:
!pip install transformers g2p_en cn2an pypinyin jieba unidecode soundfile mecab-python3 unidic-lite num2words pykakasi fugashi anyascii jamo gruut cached_path g2pkk loguru

In [7]:
%cd MeloTTS
!pip install -e .

C:\Users\JiHwan\PycharmProjects\PythonProject\MeloTTS
Obtaining file:///C:/Users/JiHwan/PycharmProjects/PythonProject/MeloTTS
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached txtsplit-1.0.0-py3-none-any.whl
  Using cached transformers-4.27.4-py3-none-any.whl.metadata (106 kB)
  Using cached num2words-0.5.12-py3-none-any.whl.metadata (11 kB)
  Using cached unidic-1.1.0-py3-none-any.whl
  Using cached mecab_python3-1.0.9-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached pykakasi-2.2.1-py3-none-any.whl.metadata (10 kB)
  Using cached fugashi-1.3.0.tar.gz (338 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached anyascii-0.3.2-py3-none-any.whl.metadata (1.5 kB)
  Using cached gruut-2.2.3-py3-none-any.whl
  Using cached librosa-0.9.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached

  DEPRECATION: Building 'fugashi' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'fugashi'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [40 lines of output]
      C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              Lice

In [8]:
!python train.py \ --config configs/base.yaml \ --exp_name myvoice \ --data_dir ./dataset

python: can't open file 'C:\\Users\\JiHwan\\PycharmProjects\\PythonProject\\MeloTTS\\train.py': [Errno 2] No such file or directory


In [2]:
import sys, subprocess
def pipi(*pkgs): subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", *pkgs])
pipi("librosa","soundfile","webrtcvad-wheels","openai-whisper","pandas","huggingface_hub")

In [5]:
# ==== 공통 유틸 ====
from pathlib import Path
import os, sys, subprocess

def P(p) -> Path:
    """무조건 Path 객체로 변환"""
    return p if isinstance(p, Path) else Path(p)

def S(p) -> str:
    """항상 절대경로 문자열로 변환 (서브프로세스/외부라이브러리용)"""
    return os.fspath(P(p).resolve())

# ==== 기본 경로 ====
ROOT = P(r"C:\Users\JiHwan\PycharmProjects\PythonProject")  # ← 필요시 수정
DATA = ROOT / "data_melo"           # 세그먼트/메타데이터 저장
EXP  = ROOT / "exp_myvoice"         # 학습 설정/체크포인트 저장
MELO = ROOT / "MeloTTS"             # MeloTTS 레포 루트

for p in [ROOT, DATA, DATA/"wavs", EXP, MELO]:
    p.mkdir(parents=True, exist_ok=True)

print("ROOT:", S(ROOT))
print("DATA:", S(DATA))
print("EXP :", S(EXP))
print("MELO:", S(MELO))


ROOT: C:\Users\JiHwan\PycharmProjects\PythonProject
DATA: C:\Users\JiHwan\PycharmProjects\PythonProject\data_melo
EXP : C:\Users\JiHwan\PycharmProjects\PythonProject\exp_myvoice
MELO: C:\Users\JiHwan\PycharmProjects\PythonProject\MeloTTS


In [6]:
import librosa, soundfile as sf

# ★ 여기만 너의 정확 파일명으로 바꿔줘 (raw string 권장)
src_audio = P(r"[사골 김치찌개] 김치CEO 홍진경이 처음 공개하는 김치찌개 레시피 (+오이지,.mp3")
assert src_audio.exists(), f"{src_audio} 파일을 찾을 수 없어요."

y, sr = librosa.load(S(src_audio), sr=24000, mono=True)
sf.write(S(ROOT/"voice_24k.wav"), y, 24000)
print("표준화 완료 →", S(ROOT/"voice_24k.wav"))


표준화 완료 → C:\Users\JiHwan\PycharmProjects\PythonProject\voice_24k.wav


In [7]:
import numpy as np, librosa, soundfile as sf, webrtcvad
from scipy.signal import resample_poly  # librosa.resample보다 빠르고 왜곡 적음

SRC_IN   = ROOT / "voice_24k.wav"
SR_IN    = 24000
SR_VAD   = 16000
FRAME_MS = 20          # ← 30 → 20ms
VAD_MODE = 1           # ← 2 → 1 (필요시 0)
PAD_SEC  = 0.5         # ← 0.2 → 0.5s
MIN_DUR  = 1.5         # ← 2.0 → 1.5s
MAX_DUR  = 12.0        # ← 10.0 → 12.0s
SR_OUT   = 24000

# 1) 로드 & 16k 변환
y24, _ = librosa.load(S(SRC_IN), sr=SR_IN, mono=True)
# 고품질 리샘플 (24k→16k): resample_poly( up=2, down=3 ) == 24k *2/3 = 16k
y16 = resample_poly(y24, 2, 3)

# 2) VAD 프레임링
vad = webrtcvad.Vad(VAD_MODE)
frame_len = SR_VAD * FRAME_MS // 1000
pad_frames = int(round(PAD_SEC * 1000 / FRAME_MS))

segments16, cur = [], []
hang = 0

def flush():
    if cur:
        segments16.append(np.concatenate(cur))
        cur.clear()

# 3) 스캔
for i in range(0, len(y16) - frame_len + 1, frame_len):
    fr = y16[i:i+frame_len]
    # int16 변환
    pcm16 = np.clip(fr * 32767, -32768, 32767).astype(np.int16).tobytes()
    if vad.is_speech(pcm16, sample_rate=SR_VAD):
        cur.append(fr); hang = pad_frames
    else:
        if hang > 0:
            cur.append(fr); hang -= 1
        else:
            flush()
flush()

# 4) 길이 필터 + 너무 긴 세그 자동 분할
def chop(seg, sr, max_dur):
    max_len = int(max_dur * sr)
    if len(seg) <= max_len:
        return [seg]
    out = []
    st = 0
    while st < len(seg):
        ed = min(st + max_len, len(seg))
        out.append(seg[st:ed])
        st = ed
    return out

paths = []
for i, seg16 in enumerate(segments16, 1):
    # MAX_DUR 초과는 분할
    for piece16 in chop(seg16, SR_VAD, MAX_DUR):
        dur = len(piece16) / SR_VAD
        if dur < MIN_DUR:   # 너무 짧은 건 스킵
            continue
        # 16k→24k 복원 (필요 없으면 그냥 16k로 저장)
        seg24 = resample_poly(piece16, 3, 2)  # 16k *3/2 = 24k
        fn = DATA/"wavs"/f"{len(paths)+1:04d}.wav"
        sf.write(S(fn), seg24, SR_OUT)
        paths.append(S(fn))

print("세그먼트 저장(24k):", len(paths))


세그먼트 저장(24k): 45


In [6]:
import soundfile as sf
info = sf.info(S(SRC_IN))
print("원본 길이(초):", info.duration)


원본 길이(초): 507.9132916666667


In [34]:
!pip install tensorflow-gpu

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Traceback (most recent call last):
        File "C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\packaging\requirements.py", line 36, in __init__
          parsed = _parse_requirement(requirement_string)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\packaging\_parser.py", line 62, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\packaging\_parser.py", line 80, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\U

In [35]:
import torch

if torch.cuda.is_available():
    print("CUDA가 다시 돌아왔습니다!")
    print(f"사용 가능한 GPU 개수: {torch.cuda.device_count()}")
else:
    print("아직 CUDA를 사용할 수 없습니다. ㅠㅠ")

아직 CUDA를 사용할 수 없습니다. ㅠㅠ


In [14]:
# GPU 확인
import sys, torch, shutil, os
print("Python:", sys.executable)
print("CUDA available:", torch.cuda.is_available())
print("torch version:", torch.__version__)
print("CUDA in torch (build):", torch.version.cuda)
print("ffmpeg:", shutil.which("ffmpeg"))
print(sys.executable)



Python: C:\Users\JiHwan\anaconda3\envs\PythonProject\python.exe
CUDA available: False
torch version: 2.8.0+cpu
CUDA in torch (build): None
ffmpeg: C:\Users\JiHwan\anaconda3\envs\PythonProject\Library\bin\ffmpeg.EXE
C:\Users\JiHwan\anaconda3\envs\PythonProject\python.exe


In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [36]:
!nvcc --version

'nvcc' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
import os, math, pandas as pd, whisper, torch

device = "cuda" if torch.cuda.is_available() else "cpu"
asr = whisper.load_model("small").to(device)
print("ASR device:", device)

rows = []
for fp in paths:
    res = asr.transcribe(fp, language="ko")
    text = res["text"].strip()
    utt_id = os.path.basename(fp).replace(".wav","")
    rows.append((utt_id, text))

df = pd.DataFrame(rows, columns=["utt_id","text"])
display(df.head())

val_n = max(1, math.floor(len(df)*0.1))
df_val = df.sample(val_n, random_state=42)
df_tr  = df.drop(df_val.index)

df_tr[["utt_id","text"]].to_csv(S(DATA/"metadata.csv"),      index=False, header=False, sep="|", encoding="utf-8")
df_val[["utt_id","text"]].to_csv(S(DATA/"metadata_val.csv"), index=False, header=False, sep="|", encoding="utf-8")

print("Train/Val:", len(df_tr), len(df_val))
print("metadata.csv:",      S(DATA/"metadata.csv"))
print("metadata_val.csv:",  S(DATA/"metadata_val.csv"))


100%|███████████████████████████████████████| 461M/461M [00:10<00:00, 48.1MiB/s]


ASR device: cpu


C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


KeyboardInterrupt: 

In [7]:
import os, math, pandas as pd, whisper, torch, soundfile as sf
from pathlib import Path

device = "cuda" if torch.cuda.is_available() else "cpu"
asr = whisper.load_model("medium").to(device)
print("ASR device:", device)

def sec(x): return int(round(x))

rows = []
dropped = {"too_short":0, "too_long":0, "empty":0}
for fp in paths:
    # 1) 실제 파일 길이 확인
    try:
        f_info = sf.info(fp)
        duration = f_info.duration
    except Exception as e:
        duration = None
    print(f"\n== {fp} ==")
    if duration is not None:
        print(f"  file_duration: {duration:.2f}s")

    # 2) 전사
    res = asr.transcribe(
        fp,
        language="ko",
        condition_on_previous_text=False,  # 세그먼트 뭉침 방지 쪽으로
        temperature=0.0,                   # 필요시 [0,0.2,0.4...] fallback 시도
    )

    segs = res.get("segments", [])
    if segs:
        print(f"  segments: {len(segs)}, last_end: {segs[-1]['end']:.2f}s")

    base = Path(fp).stem
    for i, seg in enumerate(segs):
        s, e = float(seg["start"]), float(seg["end"])
        text = (seg.get("text") or "").strip()
        dur = e - s

        # ---- (A) 우선 필터링을 완화/제거 ----
        # if dur < 2.0:
        #     dropped["too_short"] += 1; continue
        # if dur > 10.0:
        #     dropped["too_long"] += 1; continue

        if not text:
            dropped["empty"] += 1
            continue

        utt_id = f"{base}_{sec(s):06d}_{sec(e):06d}"
        rows.append((utt_id, text))

print(f"\nDropped: {dropped}")

df = pd.DataFrame(rows, columns=["utt_id","text"])
print("총 문장 수:", len(df))
display(df.head())

# Train/Val split
val_n = max(1, math.floor(len(df)*0.1))
df_val = df.sample(val_n, random_state=42)
df_tr  = df.drop(df_val.index)

df_tr[["utt_id","text"]].to_csv(S(DATA/"metadata.csv"),      index=False, header=False, sep="|", encoding="utf-8")
df_val[["utt_id","text"]].to_csv(S(DATA/"metadata_val.csv"), index=False, header=False, sep="|", encoding="utf-8")

print("Train/Val:", len(df_tr), len(df_val))
print("metadata.csv:",      S(DATA/"metadata.csv"))
print("metadata_val.csv:",  S(DATA/"metadata_val.csv"))


ASR device: cpu

== C:\Users\JiHwan\PycharmProjects\PythonProject\data_melo\wavs\0010.wav ==
  file_duration: 7.29s


C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


  segments: 1, last_end: 2.00s

== C:\Users\JiHwan\PycharmProjects\PythonProject\data_melo\wavs\0013.wav ==
  file_duration: 9.54s
  segments: 3, last_end: 9.52s

== C:\Users\JiHwan\PycharmProjects\PythonProject\data_melo\wavs\0015.wav ==
  file_duration: 4.20s
  segments: 2, last_end: 4.00s

== C:\Users\JiHwan\PycharmProjects\PythonProject\data_melo\wavs\0020.wav ==
  file_duration: 2.64s
  segments: 1, last_end: 2.60s

== C:\Users\JiHwan\PycharmProjects\PythonProject\data_melo\wavs\0026.wav ==
  file_duration: 8.82s
  segments: 6, last_end: 8.80s

== C:\Users\JiHwan\PycharmProjects\PythonProject\data_melo\wavs\0030.wav ==
  file_duration: 9.54s
  segments: 1, last_end: 2.00s

== C:\Users\JiHwan\PycharmProjects\PythonProject\data_melo\wavs\0035.wav ==
  file_duration: 2.52s
  segments: 1, last_end: 2.46s

== C:\Users\JiHwan\PycharmProjects\PythonProject\data_melo\wavs\0037.wav ==
  file_duration: 2.07s

== C:\Users\JiHwan\PycharmProjects\PythonProject\data_melo\wavs\0040.wav ==
  file

,utt_id,text
0,0010_000000_000002,음~~
1,0013_000000_000002,이게 김치찌개 끓이는 것도 다양한 방법이 있어요
2,0013_000002_000005,내가 만약에 오늘 멸치 김치찌개를 끓이겠다
3,0013_000005_000010,해산물 다시 팩에 국물 우려서 육수에다가 김치 끓여서
4,0015_000000_000002,잊어버리고 끓여야 돼.


Train/Val: 16 1
metadata.csv: C:\Users\JiHwan\PycharmProjects\PythonProject\data_melo\metadata.csv
metadata_val.csv: C:\Users\JiHwan\PycharmProjects\PythonProject\data_melo\metadata_val.csv


In [ ]:
# 7개 컬럼으로 만드는 코드
import os, math, pandas as pd, whisper, torch
# 'cleaner' 모듈을 임포트합니다.
from MeloTTS.melo.text import cleaner

device = "cuda" if torch.cuda.is_available() else "cpu"
asr = whisper.load_model("small").to(device)
print("ASR device:", device)

# 화자 ID와 언어 ID를 정의합니다.
SPEAKER_ID = "Hong"
LANGUAGE = "KR" # 언어 코드는 'KR'로 설정해야 합니다 (config.json의 'spk2id'와 일치).

rows = []
for fp in paths:
    res = asr.transcribe(fp, language="ko")
    text = res["text"].strip()
    utt_id = os.path.basename(fp).replace(".wav", "")

    # 'cleaner' 모듈의 'clean_text' 함수를 사용하여 필요한 값을 가져옵니다.
    _, phones, tones, word2ph = cleaner.clean_text(text, LANGUAGE)

    # DataFrame에 추가할 7가지 값을 올바른 순서로 정의합니다.
    rows.append((utt_id, SPEAKER_ID, LANGUAGE, text, " ".join(phones), " ".join(map(str, tones)), " ".join(map(str, word2ph))))

df = pd.DataFrame(rows, columns=["_id", "spk", "language", "text", "phones", "tone", "word2ph"])
print("생성된 데이터프레임:")
print(df.head())

val_n = max(1, math.floor(len(df) * 0.1))
df_val = df.sample(val_n, random_state=42)
df_tr = df.drop(df_val.index)

df_tr.to_csv(S(DATA / "metadata.csv"), index=False, header=False, sep="|", encoding="utf-8")
df_val.to_csv(S(DATA / "metadata_val.csv"), index=False, header=False, sep="|", encoding="utf-8")

print("훈련/검증 데이터 수:", len(df_tr), len(df_val))
print("metadata.csv 파일이 올바르게 생성되었습니다.")

C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\jieba\_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


ASR device: cpu


C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [7]:
from huggingface_hub import snapshot_download
import json, shutil

ckpt_dir = P(snapshot_download("myshell-ai/MeloTTS-Korean"))

# 워밍업 ckpt 준비
#
shutil.copy2(S(ckpt_dir/"checkpoint.pth"), S(EXP/"warmup_checkpoint.pth"))

# config 로드 + 화자 추가 + 데이터 경로 주입
cfg_path = ckpt_dir/"config.json"
if cfg_path.exists():
    hps = json.loads(cfg_path.read_text(encoding="utf-8"))
else:
    hps = {"data":{"spk2id":{"KR":0}, "n_speakers":256, "sampling_rate":24000},
           "train":{}, "model":{}}

spk2id = hps.setdefault("data", {}).get("spk2id", {"KR":0})
spk2id.setdefault("KR", 0)
spk2id["Hong"] = 1
hps["data"]["spk2id"] = spk2id
hps["data"]["n_speakers"] = max(hps["data"].get("n_speakers", 256), 2)

hps["data"]["training_files"]   = S(DATA/"metadata.csv")
hps["data"]["validation_files"] = S(DATA/"metadata_val.csv")

hps.setdefault("train", {}).update({
    "epochs": 10,
    "batch_size": 8,
    "learning_rate": 1e-4,
    "grad_accum_steps": 2,
    "mixed_precision": True,
})

cfg_out = EXP/"config_finetune_my.json"
cfg_out.write_text(json.dumps(hps, ensure_ascii=False, indent=2), encoding="utf-8")
print("spk2id:", hps["data"]["spk2id"])
print("설정 저장:", S(cfg_out))
print("워밍업 ckpt:", S(EXP/"warmup_checkpoint.pth"))


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

spk2id: {'KR': 0, 'Hong': 1}
설정 저장: C:\Users\JiHwan\PycharmProjects\PythonProject\exp_myvoice\config_finetune_my.json
워밍업 ckpt: C:\Users\JiHwan\PycharmProjects\PythonProject\exp_myvoice\warmup_checkpoint.pth


In [14]:
!pip install tensorboard
!python -m MeloTTS.melo.train

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\JiHwan\PycharmProjects\PythonProject\MeloTTS\melo\train.py", line 15, in <module>
    import commons
ModuleNotFoundError: No module named 'commons'


In [18]:
!pip install -U pip
!pip install --only-binary=:all: "mecab-python3>=1.0.8" "unidic-lite>=1.0.8"

In [33]:
!pip install --no-cache-dir --force-reinstall "numpy<2.3,>=1.24" "numba>=0.61.2"

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   --------------------------- ------------ 8.7/12.6 MB 44.9 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 46.6 MB/s  0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 2.8/2.8 MB 40.8 MB/s  0:00:00
   ---------------------------------------- 0.0/30.3 MB ? eta -:--:--
   ---- ----------------------------------- 3.1/30.3 MB 61.4 MB/s eta 0:00:01
   --------- ------------------------------ 7.3/30.3 MB 17.4 MB/s eta 0:00:02
   ------------- -------------------------- 10.2/30.3 MB 15.9 MB/s eta 0:00:02
   ----------------------------- ---------- 22.3/30.3 MB 26.6 MB/s eta 0:00:01
   ---------------------------------------- 30.3/30.3 MB 31.0 MB/s  0:00:00

  Attempting uninstall: numpy

    Found existing installation: numpy 2.2.6

    Uninstalling numpy-2.2.6:

   ---------------------------------------- 0/3 [numpy]
   -----

In [30]:
!chcp 65001
!set PYTHONUTF8=1
!set PYTHONIOENCODING=utf-8

Active code page: 65001


In [31]:
import numpy
numpy.__version__

'2.2.6'

In [32]:
!python - << "PY"
import numpy, transformers
print("numpy:", numpy.__version__, "|", numpy.__file__)
print("transformers:", transformers.__version__)
!PY

<< was unexpected at this time.
C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


numpy: 2.2.6 | C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\numpy\__init__.py
transformers: 4.55.4


'PY' is not recognized as an internal or external command,
operable program or batch file.


In [33]:
# 모델 허용값 확인
!python -m MeloTTS.melo.train --help

[WARN] CUDA 미사용: CPU로 학습합니다(매우 느릴 수 있음).
usage: train.py [-h] [-c CONFIG] [--local_rank LOCAL_RANK]
                [--world-size WORLD_SIZE] [--port PORT] -m MODEL
                [--pretrain_G PRETRAIN_G] [--pretrain_D PRETRAIN_D]
                [--pretrain_dur PRETRAIN_DUR]

options:
  -h, --help            show this help message and exit
  -c CONFIG, --config CONFIG
                        JSON file for configuration
  --local_rank LOCAL_RANK
  --world-size WORLD_SIZE
  --port PORT
  -m MODEL, --model MODEL
                        Model name
  --pretrain_G PRETRAIN_G
                        pretrain model
  --pretrain_D PRETRAIN_D
                        pretrain model D
  --pretrain_dur PRETRAIN_DUR
                        pretrain model duration


C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\jieba\_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [14]:
%cd PycharmProjects\PythonProject

C:\Users\JiHwan\PycharmProjects\PythonProject


In [30]:
import requests

url = "https://myshell-public-repo-host.s3.amazonaws.com/openvoice/basespeakers/pretrained/G.pth"
r = requests.head(url)
print("Status:", r.status_code)


Status: 200


In [35]:
from MeloTTS.melo.download_utils import PRETRAINED_MODELS, load_pretrain_model
local_paths = load_pretrain_model()  # 자동으로 다운로드 혹은 캐시 경로 반환
print(local_paths)  # [G.pth 경로, D.pth 경로, DUR.pth 경로]


[WindowsPath('C:/Users/JiHwan/.cache/cached_path/73ad3d5a37c82356ed81630b0a435b4b376ca49523854fe2b8302609fd71c193.133b77b9d9162e348486a0a0778fa47d726930e3ec12ea5e2684c0c919743a65'), WindowsPath('C:/Users/JiHwan/.cache/cached_path/c3d3c787a8711093a79ee95f091a35de75e527b6e8e28424ad7010f6e86cce58.e5f88bb1eca17c37beb511b15a932e84fdc8b66d8a8d5c5075334650425954f2'), WindowsPath('C:/Users/JiHwan/.cache/cached_path/c7b373ab8939eb672a985a802d21420534ca0cd43fa4aecf4fa6088a569ee2a1.ce7a8153914d9727ebc28e4b4e3d31eed35aa0b4e3d125eb54e6f8363968dd7a')]


In [36]:
import os
p = r"C:\Users\JiHwan\PycharmProjects\PythonProject\exp_myvoice\G.pth"
print(os.path.exists(p), os.path.getsize(p) if os.path.exists(p) else None)


True 207784400


In [ ]:
# 기존의 '모델훈련' 셀을 이 코드로 대체합니다.
# 이 셀은 MeloTTS/melo/train.py 파일의 기능을 직접 호출하는 방식입니다.

# 필요한 모듈 임포트
import subprocess
import sys
import os
import torch
from pathlib import Path

# 기존 경로 변수들이 잘 정의되어 있다고 가정
# ROOT, EXP, MELO 등

# 환경 변수 설정
os.environ['TORCH_USE_LIBUV'] = "0"
os.environ['TP_SOCKET_USE_LIBUV'] = "0"
os.environ['GLOO_USE_LIBUV'] = "0"
os.environ['MELO_DISABLE_DDP'] = "1"

# sys.path에 MeloTTS 레포지토리 루트 추가
if S(MELO) not in sys.path:
    sys.path.append(S(MELO))

print("sys.path에 MeloTTS 추가 완료.")
print("환경 변수 설정 완료.")

# MeloTTS.melo.train 모듈을 임포트하고 main 함수 직접 호출
# Note: 이 방법은 train.py의 내부 구조에 따라 성공 여부가 달라질 수 있습니다.
# 만약 작동하지 않는다면, 두 번째 해결책을 시도해야 합니다.
try:
    from melo.train import main as train_main

    # main 함수에 필요한 인자 설정
    parser_args = [
        "--model", "vits2",
        "--config", S(EXP/"config_finetune_my.json"),
        "--pretrain_G", S(EXP/"warmup_checkpoint.pth"),
        "--world-size", "1"
    ]

    print("직접 train.py main 함수 호출 시도...")
    train_main(parser_args)
    print("훈련 스크립트 실행 완료.")

except ImportError as e:
    print(f"MeloTTS.melo.train 모듈을 찾을 수 없습니다. 에러: {e}")
    print("직접 실행 대신 subprocess 방식을 사용합니다.")

    # 두 번째 해결책: subprocess.run()을 사용하여 더 명시적인 실행
    cmd = [
        sys.executable,  # 현재 사용 중인 파이썬 인터프리터
        "-m", "MeloTTS.melo.train",
        "--model", "vits2",
        "--config", S(EXP/"config_finetune_my.json"),
        "--pretrain_G", S(EXP/"warmup_checkpoint.pth"),
        "--world-size", "1"
    ]

    print("\nsubprocess로 훈련 스크립트 실행 시작...")
    print("명령어:", " ".join(cmd))

    process = subprocess.Popen(cmd,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.STDOUT,
                               universal_newlines=True,
                               encoding="utf-8",
                               cwd=S(ROOT))

    # 실시간으로 출력 로그를 표시
    for line in iter(process.stdout.readline, ''):
        sys.stdout.write(line)

    process.wait()
    print("subprocess 실행 완료, 종료 코드:", process.returncode)

sys.path에 MeloTTS 추가 완료.
환경 변수 설정 완료.
MeloTTS.melo.train 모듈을 찾을 수 없습니다. 에러: No module named 'melo.train'
직접 실행 대신 subprocess 방식을 사용합니다.

subprocess로 훈련 스크립트 실행 시작...
명령어: C:\Users\JiHwan\anaconda3\envs\PythonProject\python.exe -m MeloTTS.melo.train --model vits2 --config C:\Users\JiHwan\PycharmProjects\PythonProject\exp_myvoice\config_finetune_my.json --pretrain_G C:\Users\JiHwan\PycharmProjects\PythonProject\exp_myvoice\warmup_checkpoint.pth --world-size 1
C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\jieba\_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or chars

In [19]:
# (선택) 혹시 모를 libuv 경로 완전 차단
!env:TORCH_USE_LIBUV="0"
!env:TP_SOCKET_USE_LIBUV="0"
!env:GLOO_USE_LIBUV="0"

# 분산 비활성화 신호 (아래 코드 패치도 함께 적용할 때 효과적)
!env:MELO_DISABLE_DDP="1"

!python -m MeloTTS.melo.train `-m vits2 `-c C:\Users\JiHwan\PycharmProjects\PythonProject\exp_myvoice\config_finetune_my.json `--pretrain_G C:\Users\JiHwan\PycharmProjects\PythonProject\exp_myvoice\warmup_checkpoint.pth `--world-size 1


The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.
Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 159, in _get_module_details
  File "<frozen importlib._bootstrap_external>", line 1133, in get_code
  File "<frozen importlib._bootstrap_external>", line 1063, in source_to_code
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "C:\Users\JiHwan\PycharmProjects\PythonProject\MeloTTS\melo\train.py", line 265
    global global_step
    ^^^^^^^^^^^^^^^^^^
SyntaxError: name 'global_step' is assigned to before global declaration


In [10]:
p = subprocess.run(cmd, env=env, capture_output=True, text=True)
print("=== STDOUT ===")
print(p.stdout)
print("=== STDERR ===")
print(p.stderr)
print("returncode:", p.returncode)


=== STDOUT ===

=== STDERR ===
C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\jieba\_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_mo

In [3]:
torch.__version__

'2.8.0+cpu'

In [ ]:
import re, torch
from pathlib import Path

# EXP 폴더 우선으로 검색, 없으면 MeloTTS 하위도 스캔
cands = sorted(EXP.rglob("*.pth"), key=os.path.getmtime)
cands = [p for p in cands if not re.search(r"(warmup|optimizer|ema|baseline)", p.name, re.I)]

if not cands:
    cands = sorted((MELO).rglob("*.pth"), key=os.path.getmtime)
    cands = [p for p in cands if not re.search(r"(warmup|optimizer|ema|baseline)", p.name, re.I)]

print("후보 ckpt 최근 5개:", [S(p) for p in cands[-5:]])
assert cands, "학습된 .pth를 못 찾았어요. 저장 경로/주기를 확인하세요."
finetuned_ckpt = cands[-1]
print("사용 ckpt:", S(finetuned_ckpt))

# 합성
sys.path.append(S(MELO))
from melo.api import TTS

device = "cuda" if torch.cuda.is_available() else "cpu"
tts = TTS(language="KR", device=device, ckpt_path=S(finetuned_ckpt))
spk2id = tts.hps.data.spk2id
print("등록 화자맵:", spk2id)

spk = spk2id.get("Hong", spk2id.get("KR", next(iter(spk2id.values()))))
out_wav = ROOT/"myvoice_sample.wav"
tts.tts_to_file("안녕하세요, 홍진경입니다. 솔로지옥 많은 관심 부탁드려요.", spk, S(out_wav), speed=1.0)
print("합성 출력:", S(out_wav))
